## Google authentication

In [1]:
import os
import tensorflow as tf
import json
from google.colab import auth
auth.authenticate_user()

use_tpu = 'COLAB_TPU_ADDR' in os.environ
if use_tpu:
  TPU_ADDRESS = 'grpc://' + os.environ['COLAB_TPU_ADDR']
  print('TPU address is', TPU_ADDRESS)
  
  with tf.Session(TPU_ADDRESS) as session:
    # Upload credentials to TPU.
    with open('/content/adc.json', 'r') as f:
      auth_info = json.load(f)
    tf.contrib.cloud.configure_gcs(session, credentials=auth_info)
    # Now credentials are set for all future sessions on this TPU.
else:
  device_name = tf.test.gpu_device_name()
  if device_name != '/device:GPU:0':
    raise SystemError('GPU device not found')
  print('Found GPU at: {}'.format(device_name))

TPU address is grpc://10.83.54.162:8470


## Download *improv* project

In [2]:
import sys

!test -d improv || git clone https://github.com/rikhuijzer/improv improv
if not 'improv' in sys.path:
  sys.path += ['improv']

Cloning into 'improv'...
remote: Enumerating objects: 422, done.
remote: Counting objects: 100% (422/422), done.
remote: Compressing objects: 100% (266/266), done.
remote: Total 422 (delta 261), reused 306 (delta 149), pack-reused 0
Receiving objects: 100% (422/422), 3.01 MiB | 9.70 MiB/s, done.
Resolving deltas: 100% (261/261), done.


## BERT pre-trained 


In [0]:
bucket = 'benchmark-tpu-bucket' #@param {type:"string"}
assert bucket, 'Must specify an existing GCS bucket name'
bucket_models_dir = 'gs://{}/bert/models'.format(bucket)
bert_model = 'BERT-Base, Uncased' #@param ['BERT-Base, Uncased', 'BERT-Large, Uncased']
bert_model_map = {
    'BERT-Base, Uncased': 'uncased_L-12_H-768_A-12',
    'BERT-Large, Uncased': 'uncased_L-24_H-1024_A-16',
}
bert_model_mapped = bert_model_map[bert_model]

bert_pretrained_dir = 'gs://cloud-tpu-checkpoints/bert/' + bert_model_mapped

## Define params

In [4]:
!pip install rasa_nlu

    100% |████████████████████████████████| 153kB 5.6MB/s 
    100% |████████████████████████████████| 51kB 8.1MB/s 
    100% |████████████████████████████████| 81kB 6.2MB/s 
    100% |████████████████████████████████| 51kB 8.6MB/s 
    100% |████████████████████████████████| 133kB 5.3MB/s 
    100% |████████████████████████████████| 4.5MB 4.9MB/s 
    100% |████████████████████████████████| 81kB 25.6MB/s 
    100% |████████████████████████████████| 5.1MB 6.7MB/s 
    100% |████████████████████████████████| 61kB 23.6MB/s 
    100% |████████████████████████████████| 51kB 20.5MB/s 
    100% |████████████████████████████████| 3.1MB 9.6MB/s 
    100% |████████████████████████████████| 552kB 26.3MB/s 
    100% |████████████████████████████████| 174kB 29.9MB/s 
    100% |████████████████████████████████| 61kB 23.3MB/s 
  Running setup.py bdist_wheel for pathlib ... - done
  Stored in directory: /root/.cache/pip/wheels/f9/b2/4a/68efdfe5093638a9918bd1bb734af625526e849487200aa171
  Running se

In [0]:
from src.config import HParams
from src.utils import get_project_root
from pathlib import Path
from datetime import datetime, timedelta

task_name = 'askubuntu'  #@param {type:"string"}
epochs = 60  #@param {type:"integer"}
output_dir_name = str(datetime.now() + timedelta(hours=1))[:-7]
tpu_name = 'grpc://' + os.environ['COLAB_TPU_ADDR'] if use_tpu else ''

hparams = HParams(
  data_dir=get_project_root() / 'data' / task_name,
  bert_config_file=os.path.join(bert_pretrained_dir, 'bert_config.json'),
  task_name=task_name,
  vocab_file=os.path.join(bert_pretrained_dir, 'vocab.txt'),
  output_dir=bucket_models_dir + '/{}/{}'.format(task_name, output_dir_name),
  init_checkpoint=os.path.join(bert_pretrained_dir, 'bert_model.ckpt'),
  do_lower_case=bert_model.startswith('uncased'),
  max_seq_length=128,
  do_train_eval=False,
  do_train=True,
  do_eval=False,
  do_predict=True, 
  train_batch_size=32,
  eval_batch_size=8,
  predict_batch_size=8,
  learning_rate=5e-5,
  num_train_epochs=epochs,
  warmup_proportion=0.1,
  save_checkpoints_steps=1000,
  save_summary_steps=8,
  iterations_per_loop=1000,  # ignored 
  use_tpu=use_tpu,
  tpu_name=tpu_name,
  tpu_zone=None,
  gcp_project=None,
  master=None,
  num_tpu_cores=8
)

tf.gfile.MakeDirs(hparams.output_dir)

## Preparation


In [6]:
from src.my_classifier import (
    get_model_fn_and_estimator, evaluate, train, predict
)

tf.gfile.MakeDirs(str(hparams.output_dir))

model_fn, estimator = get_model_fn_and_estimator(hparams)

INFO:tensorflow:Using config: {'_model_dir': 'gs://benchmark-tpu-bucket/bert/models/askubuntu/2018-11-29 15:20:17', '_tf_random_seed': None, '_save_summary_steps': 8, '_save_checkpoints_steps': 1000, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
cluster_def {
  job {
    name: "worker"
    tasks {
      value: "10.83.54.162:8470"
    }
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f6512e07dd8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': b'grpc://10.83.54.162:8470', '_evaluation_master': b'grpc://10.83.54.162:8470', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1, '_tpu_config': TPUConfig(iterations_per_loop=1000, num_sh

## Train

In [0]:
training_start_time = datetime.now()

In [8]:
from src.my_classifier import train

if hparams.do_train:
  train(hparams, estimator)

INFO:tensorflow:Writing example 0 of 53
INFO:tensorflow:*** Example ***
INFO:tensorflow:guid: train-0
INFO:tensorflow:tokens: [CLS] [UNK] to record my screen ? [SEP]
INFO:tensorflow:input_ids: 101 100 2000 2501 2026 3898 1029 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
INFO:tensorflow:input_mask: 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
INFO:tensorflow:segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

In [9]:
duration = str(datetime.now() - training_start_time)
print('Training took {}.'.format(duration))

Training took 0:02:39.662580.


## Eval

In [0]:
if hparams.do_eval:
  evaluate(hparams, estimator)

## Predict

In [11]:
# note that predictions are non-deterministic
if hparams.do_predict:
  predict(hparams)

INFO:tensorflow:Using config: {'_model_dir': 'gs://benchmark-tpu-bucket/bert/models/askubuntu/2018-11-29 15:20:17', '_tf_random_seed': None, '_save_summary_steps': 8, '_save_checkpoints_steps': 1000, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
cluster_def {
  job {
    name: "worker"
    tasks {
      value: "10.83.54.162:8470"
    }
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f651221ec50>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': b'grpc://10.83.54.162:8470', '_evaluation_master': b'grpc://10.83.54.162:8470', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1, '_tpu_config': TPUConfig(iterations_per_loop=1000, num_sh

In [12]:
from src.my_estimator import predict
from src.utils import get_y_true

y_pred = predict(hparams)
y_true = get_y_true(hparams.data_dir / 'askubuntu.tsv', training=False)

print('y_pred', 'y_true')
for pred, true in zip(y_pred, y_true):
  print(pred, ', ', true)

INFO:tensorflow:Using config: {'_model_dir': 'gs://benchmark-tpu-bucket/bert/models/askubuntu/2018-11-29 15:20:17', '_tf_random_seed': None, '_save_summary_steps': 8, '_save_checkpoints_steps': 1000, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
cluster_def {
  job {
    name: "worker"
    tasks {
      value: "10.83.54.162:8470"
    }
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f65049e95c0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': b'grpc://10.83.54.162:8470', '_evaluation_master': b'grpc://10.83.54.162:8470', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1, '_tpu_config': TPUConfig(iterations_per_loop=1000, num_sh